In [ ]:
import json
import numpy as np

with open('knu/SentiWord_info.json', encoding='utf-8-sig', mode='r') as f:
    data = json.load(f)
    
data_list = [list(data[i].values()) for i in range(len(data))]
data_array = np.asarray(data_list)

word = np.char.strip(data_array[:,0])
polarity = data_array[:,2]

In [ ]:
def text2array(text):
    from konlpy.tag import Okt
    import numpy as np
    
    okt = Okt()
    lines = text.split("\n")
    results = []
    for line in lines:
        wlist = okt.pos(line)
        r = []
        for w in wlist:
            if not w[1] in ['Josa', 'Eomi', 'Punctuation']:
                r.append(w[0])

        rl = (" ".join(r)).strip()
        results.append(rl)

    res = " ".join(results)
    res = res.split()
    comb = ["".join(res[m:n+1]) for m in range(len(res)) for n in range(m,m+2) if n < len(res)]
    comb = np.asarray(comb)
    
    return comb

In [ ]:
import os
import calendar
from pathos.multiprocessing import ProcessingPool as Pool
import pandas as pd
import time

cal = calendar.Calendar()

qlist = ["여성", "여자+-여성", "남성", '남자+-남성']

start = time.time()
print("hello")

for query in qlist:
    query1 = "#2_" + query
    query2 = "#3_" + query
    dir1 = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', "#3_knu_score", query2)
    if not os.path.exists(dir1):
        os.makedirs(dir1)

    for year in range(2000, 2020):
        dir2 = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', "#3_knu_score", query2, 
                       '#3_'+ str(year) + '_'+query)
        if not os.path.exists(dir2):
            os.makedirs(dir2)

        for month in range(1, 13):
                monthdays = [d for d in cal.itermonthdays(year, month) if d != 0]
                for day in monthdays:

                    date1 = str(year) + "." + str(month).zfill(2) + "." + str(day).zfill(2)
                    date2 = date1.replace(".","")
                    print(date1)

                    xlsx_name1 = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', 
                                              "#2_html2text", query1, '#2_'+ str(year) + '_'+query, 
                                             '{}.xlsx'.format("#2_"+date2+"_"+query))

                    xlsx_name2 = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', 
                                              '#3_knu_score', query2, '#3_'+ str(year) + '_'+query, 
                                             '{}.xlsx'.format("#3_"+date2+"_"+query))

                    df = pd.read_excel(xlsx_name1, index_col=0)
                    SCORE = list()

                    for i in range(0, len(df), 10):

                        tlist = list()
                        for j in range(10):
                            if i + j == len(df):
                                break
                            tlist.append(df.loc[i+j, 'text'])

                        with Pool(10) as p:
                            clist = p.map(text2array, tlist)

                        for comb in clist:
                            true_false = np.in1d(word, comb)
                            t_score = polarity[true_false]
                            if t_score.size:
                                score = sum(t_score.astype(int))/len(t_score)
                                SCORE.append(score)
                            else:
                                SCORE.append(0)

                    df['knu_score'] = SCORE
                    df.to_excel(xlsx_name2)

end = time.time()
print(end - start)